## Visualizing Oceans 2.0

---

For motivation, from [Ocean Networks Canada](http://www.oceannetworks.ca):

>"Long-term, continuous scientific data from the ocean environment are gathered by Ocean Networks Canada and made available through **Oceans 2.0**—a powerful online data management system. Oceans 2.0, combined with high-performance computing, allows ONC to provide ocean analytics that assist researchers, communities, industry, and policy-makers in making evidence-based decisions in Canada and globally."

---

Let's make a request to $\text{Oceans} \ 2.0$.

In [1]:
import requests

In [2]:
url = 'https://data.oceannetworks.ca/api/locations'
parameters = {
    'method':'get', # we want to "get" data
    'token':'f740427d-d987-43a4-b339-ea8451195bdb' # we need an access token to make requests
}

In [3]:
response = requests.get(url, params=parameters)

We may check that the `response` is well-received.

In [4]:
response.ok

True

In [5]:
response.status_code 

200

$\uparrow$ We should also be reluctant to move on if this status code is anything _but_ $200$.

---

The data we receive has a [$\text{JSON}$](https://www.json.org/) format, so let's import a module for that too.

In [6]:
import json

Extracting locations ... `json.loads` should give us a `list`.

In [7]:
locations = str(response.content, 'utf-8')
type(json.loads(locations)) # just checking ...

list

We may [slice](https://en.wikipedia.org/wiki/Array_slicing) the first $2$ elements of `locations` for a preview.

In [8]:
json.loads(locations)[:2]

[{'bbox': None,
  'dataSearchURL': 'http://data.oceannetworks.ca/DataSearch?location=ALBH',
  'deployments': 2,
  'depth': None,
  'description': 'Description:Albert Head (Metchosen) is one of the Earthquake Early Warning sites. It has a latitude of 48.3897 and a longitude of -123.4875 with an elevation of approximately 49 metres. ',
  'hasDeviceData': 'true',
  'hasPropertyData': 'false',
  'lat': None,
  'locationCode': 'ALBH',
  'locationName': 'Albert Head',
  'lon': None},
 {'bbox': {'maxDepth': -0.066956,
   'maxLat': 48.584709,
   'maxLon': -123.517527,
   'minDepth': -0.066956,
   'minLat': 48.584709,
   'minLon': -123.517527},
  'dataSearchURL': 'http://data.oceannetworks.ca/DataSearch?location=ARG',
  'deployments': 6,
  'depth': -0.066956,
  'description': 'Platform: Remotely Operated Vehicle operated by Ocean Exploration Trust. It is a support vehicle to Hercules.  ',
  'hasDeviceData': 'true',
  'hasPropertyData': 'false',
  'lat': 48.584709,
  'locationCode': 'ARG',
  'lo

---

So that we may handle our newfound `data`, let's drop this into a `DataFrame` via [`pandas`](https://pandas.pydata.org/).

In [9]:
import pandas

In [10]:
data = pandas.read_json(locations)
type(data) # another check ...

pandas.core.frame.DataFrame

In [11]:
data[:3] # a slice of 3 elements

,bbox,dataSearchURL,deployments,depth,description,hasDeviceData,hasPropertyData,lat,locationCode,locationName,lon
0,None,http://data.oceannetworks.ca/DataSearch?locati...,2,NaN,Description:Albert Head (Metchosen) is one of ...,true,false,NaN,ALBH,Albert Head,NaN
1,"{'maxDepth': -0.066956, 'maxLat': 48.584709, '...",http://data.oceannetworks.ca/DataSearch?locati...,6,-0.066956,Platform: Remotely Operated Vehicle operated b...,true,false,48.584709,ARG,Argus,-123.517527
2,"{'maxDepth': 115.0, 'maxLat': 48.300967, 'maxL...",http://data.oceannetworks.ca/DataSearch?locati...,27,112.555556,Depth: 121 m Latitude:48.300556 Longitude: -12...,true,false,48.300638,AS04,AS04 Mooring,-123.391084


There's quite a bit of information here.

In [12]:
len(data) # the "length" of the DataFrame

389

In [13]:
data.columns.values

array(['bbox', 'dataSearchURL', 'deployments', 'depth', 'description',
       'hasDeviceData', 'hasPropertyData', 'lat', 'locationCode',
       'locationName', 'lon'], dtype=object)

For now, let's just see where our `data` is coming from.

In [14]:
latitudes = data['lat']
latitudes[:3]

0          NaN
1    48.584709
2    48.300638
Name: lat, dtype: float64

In [15]:
longitudes = data['lon']
longitudes[:3]

0           NaN
1   -123.517527
2   -123.391084
Name: lon, dtype: float64

It seems that some locations don't have a registered latitude nor longitude.

For _now_, we exclude these and zip `latitudes` with `longitudes`.

In [16]:
# a quick way to catch NaNs in DataFrames: check if one equals itself
coordinates = list(zip(
    [l for l in latitudes if l == l],
    [l for l in longitudes if l == l]
))
coordinates[:4] # one more slice of 4 elements

[(48.584708999999997, -123.517527),
 (48.300637999999999, -123.39108400000001),
 (48.316685, -126.050355),
 (48.309823000000002, -126.06212499999999)]

---

Here $\mathbf{D3}.js$ will give us a better view.

In [17]:
%%javascript
require.config({ paths: { d3: "//d3js.org/d3.v4.min" } }); // "import" D3
require(["d3"], function(d3) { window.d3 = d3; });

<IPython.core.display.Javascript object>

In [18]:
width = 960; height = 300 # set SVG parameters, Python-side
svg = '<svg width = "%s" height = "%s"></svg>' % (width, height)

In [19]:
from IPython.display import HTML, Javascript; HTML(svg)

A [Mercator projection](https://en.wikipedia.org/wiki/Mercator_projection) will do.

We need to pass the `coordinates` to $\text{JavaScript}$.

In [20]:
# we cheat with a global declaration
Javascript("window.coordinates = %s" % json.dumps({'coordinates': coordinates}))

<IPython.core.display.Javascript object>

Here's the visualization code, applying one of $\text{Jupyter's}$ [_magics_](http://ipython.readthedocs.io/en/stable/interactive/magics.html).

In [21]:
%%javascript
var svg = d3.select("svg"),
    width = +svg.attr("width"),
    height = +svg.attr("height");
    
var projection = d3.geoMercator(), // Mercator projection
    path = d3.geoPath()
        .projection(projection);

var url = ("https://gist.githubusercontent.com/ericeasthope/d4e52335899a37f825b10053ba8801dc" +
           "/raw/ac330f677a117cc12ed63a56fee283cb0a5d6667/north-america.geojson");
d3.json(url, function(err, geojson) {
    svg.append("path")
       .attr("d", path(geojson));
    svg.selectAll("circle")
           .data(coordinates.coordinates).enter()
       .append("circle")
	   .attr("cx", d => projection(d.reverse())[0])
       .attr("cy", d => projection(d)[1])
	   .attr("r", "2px")
       .attr("fill", "red");
})

<IPython.core.display.Javascript object>

$\uparrow\uparrow$ Look above! The <font color="red">red</font> dots are the locations within our `data`.

---

* _Assembled by Eric Easthope_

* I humbly and _**heavily**_ borrow bits of [$\mathbf{D3}$](https://d3js.org/) from [bl.ocks](https://bl.ocks.org/) and [Stack Overflow](https://stackoverflow.com/).

---